<a href="https://colab.research.google.com/github/bennng/Spark/blob/master/spark_lr_ecom_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark Linear Regression Demonstration

In [0]:
import findspark
findspark.init('/home/ben/spark-3.0.0-preview-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('linear_regression_ecom').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True, header=True)

In [0]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [0]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [0]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [0]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [0]:
output = assembler.transform(data)

In [0]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [0]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [0]:
final_data = output.select("features",'Yearly Amount Spent')
train_data,test_data = final_data.randomSplit([0.75,0.25])
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                368|
|   mean|  495.9653900183871|
| stddev|  82.49528108150388|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [0]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                132|
|   mean|  508.6496636555225|
| stddev|  69.12139916358014|
|    min|  319.9288698031936|
|    max|   666.125591725772|
+-------+-------------------+



In [0]:
lr = LinearRegression(labelCol='Yearly Amount Spent')
lrModel = lr.fit(train_data,)
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.920172248886644,39.04170734152336,0.44862316584619216,61.7440505988246] Intercept: -1062.0839051422186


In [0]:
test_results = lrModel.evaluate(test_data)
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-11.547741616546432|
| 10.209567087504752|
| -4.819121524900481|
|-22.072495066624356|
|   9.42507464224036|
| -4.183244537577821|
| 3.3195640241191313|
| 0.5040002289524637|
|-4.8852979575760855|
|-2.7529455052546155|
| -5.351948721453766|
|  7.428932267044672|
| 0.9123529366950152|
| -5.699374093433505|
|-14.179693679654747|
|-0.2107830020809729|
| -9.230193090986404|
|  3.035489823709156|
|  4.348119829620487|
| -6.748740298909524|
+-------------------+
only showing top 20 rows



In [0]:
for item in data.head():unlabeled_data = test_data.select('features')

In [0]:
predictions = lrModel.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|331.47661141974004|
|[30.7377203726281...|451.57117510872513|
|[30.8364326747734...| 472.3210219518901|
|[31.1239743499119...| 509.0195489063901|
|[31.1695067987115...|417.93145616005245|
|[31.2681042107507...|427.65377771140174|
|[31.3584771924370...| 491.8563864253563|
|[31.3895854806643...| 409.5656108310304|
|[31.4252268808548...|  535.652016612338|
|[31.5761319713222...|  543.979529494583|
|[31.7656188210424...| 501.9060303570609|
|[31.8209982016720...| 417.2463487461687|
|[31.8293464559211...|   384.23998505128|
|[31.8745516945853...|  397.984618339701|
|[31.9365486184489...|441.37907857498294|
|[32.0047530203648...| 463.9567641227104|
|[32.0085045178551...| 452.4274141197418|
|[32.0123007682454...| 489.9095632422491|
|[32.0305497162129...| 589.9263635889913|
|[32.0883806304482...| 518.9146066870676|
+--------------------+------------

In [0]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.968185033455288
MSE: 99.36471286120201
